## Question 1 
####  If a decision tree is under-fitting the training dataset, is it a good idea to try scaling the input features?
It is not a good idea because decision tree is not sensible and does not depend on scaled input features.

## Question 2
#### If a decision tree is over-fitting the training dataset, is it a good idea to try decreasing max depth?
It is a good idea because the deeper the tree is, more complex the model becomes. Therefore, if a decision tree is over-fitting the training dataset, you want to decrease the max_depth, it would increase the tranning error, but it also allows the model to indentify important patterns.

## Question 3
#### Why would you use a random forest instead of a decision tree?
#### (a) For a lower training error.
#### (b) To reduce the variance of the model.
#### (c) For a model that it is easier for human to interpret.

---> (b) To reduce the variance of the model.

## Question 4
#### Which of the following is/are TRUE about bagging trees?
#### (a) In bagging trees, the trees are grown independent of each other.
#### (b) In bagging trees, the trees are grown in sequence. 1
#### (c) Bagging is a method for improving the performance by aggregating the results of weak learners.
#### (d) (a) and (c)
#### (e) (b) and (c)
#### (f) None of the above.

---> (d) (a) and (c)

In [1]:
## Question 6
#
3(a)
import boto3
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score

# Defining s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'gabrielferreira-data-455-bucket'
bucket = s3.Bucket(bucket_name)

# Defining the file to be read from s3 bucket
key_file = 'framingham.csv'

bucket_object = bucket.Object(key_file)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# Reading csv
heart = pd.read_csv(file_content_stream)

# (b)
heart = heart.dropna()
heart.head(1)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0


In [15]:
# (c)
# Defining input and target variables
X = heart.drop(columns = 'TenYearCHD')
Y = heart['TenYearCHD']

# Computing feature importance
feature_importance = list()

# Creating a foor loop to store 100 feature importances
for i in range(0,100):
    # Splitting the data into train and test
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    # Random Forest model variable selector
    rf_md = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)
    
    # Computing variable importance for each variable
    feature_importance.append(rf_md.feature_importances_)

# Seeing results
feature_importance = pd.DataFrame(feature_importance, columns = X.columns)
feature_importance = np.mean(feature_importance, axis = 0).sort_values(ascending = False)
feature_importance

sysBP              0.135187
BMI                0.127211
age                0.124732
totChol            0.121976
glucose            0.119477
diaBP              0.119008
heartRate          0.096263
cigsPerDay         0.050504
education          0.036787
male               0.021138
prevalentHyp       0.018285
currentSmoker      0.012654
BPMeds             0.006994
diabetes           0.006561
prevalentStroke    0.003223
dtype: float64

In [23]:
# (d)
# Defining input variables according to Random Forest feature importance
X = heart[['sysBP', 'BMI', 'age', 'totChol', 'glucose']]
Y = heart['TenYearCHD']

# Computing recall score for each model
recall_3 = list()
recall_5 = list()
recall_7 = list()

# Creating a foor loop to store 100 recall score provided by the different max_depth
for i in range(0, 100):
    # Splitting the data into train and test using new predictors 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ### Random Forest Classifier model with max_depth = 3
    rf_md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    
    # Predicting on test dataset
    pred1 = rf_md1.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 >= 0.1, 1, 0)
    
    # Computing recall
    recall_3.append(recall_score(Y_test, pred1))
    
    ### Random Forest Classifier model with max_depth = 5
    rf_md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)
    
    # Predicting on test dataset
    pred2 = rf_md2.predict_proba(X_test)[:, 1]
    pred2 = np.where(pred2 >= 0.1, 1, 0)
    
    # Computing recall
    recall_5.append(recall_score(Y_test, pred2))
    
    ### Random Forest Classifier model with max_depth = 7
    rf_md3 = RandomForestClassifier(n_estimators = 500, max_depth = 7).fit(X_train, Y_train)
    
    # Predicting on test dataset
    pred3 = rf_md3.predict_proba(X_test)[:, 1]
    pred3 = np.where(pred3 >= 0.1, 1, 0)
    
    # Computing recall
    recall_7.append(recall_score(Y_test, pred3))
    
# Seeing the average of the 100 interactions
recall_avg = {"Recall_3": [np.mean(recall_3, axis = 0)], "Recall_5": [np.mean(recall_5, axis = 0)], "Recall_7": [np.mean(recall_7, axis = 0)]}
recall_avg = pd.DataFrame(recall_avg)
recall_avg.head()

,Recall_3,Recall_5,Recall_7
0,0.844554,0.830536,0.817143


#### I would use the Random Forest Classifier using 500 trees and maximum depth tree equal to 3 because it is the simplest model and has the highest recall score between the three models.